In [15]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Load data from aac_shelter_outcomes.csv
df = pd.read_csv('aac_shelter_outcomes.csv')

if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################

app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Div([
        html.Img(src='/assets/Grazioso_Salvare_Logo.png', style={'height': '50px', 'marginRight': '10px'}),
        html.B(html.H1('Juan Matos Project 2', style={'display': 'inline-block', 'margin': 0}))
    ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center'}),
    html.Hr(),
        
    html.Div([
        html.H4("Rescue Type Filter"),
        dcc.RadioItems(
            id='rescue-type-filter',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain Rescue', 'value': 'Mountain Rescue'},
                {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
                {'label': 'Reset Filters', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )
    ]),
    html.Hr(),
    
    html.Div([
        html.H4("Outcome Type Filter"),
        dcc.Dropdown(
            id='outcome-type-filter',
            options=[
                {'label': 'Adoption', 'value': 'Adoption'},
                {'label': 'Transfer', 'value': 'Transfer'},
                {'label': 'Return to Owner', 'value': 'Return to Owner'},
                {'label': 'Euthanasia', 'value': 'Euthanasia'},
                {'label': 'All', 'value': 'All'}
            ],
            value='All',
            clearable=False,
            searchable=False,
            style={'width': '50%'}
        )
    ]),
    html.Hr(),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        selected_rows=[0],
        row_selectable='single',
        style_cell={'textAlign': 'left', 'minWidth': '100px', 'width': '150px', 'maxWidth': '200px'},
        style_table={'overflowX': 'auto'},
    ),
    
    html.Br(),
    html.Hr(),

    html.Div(className='row', style={'display': 'flex'}, children=[
        # Bar chart
        html.Div(id='graph-id', className='col s12 m6'),

        # Geolocation map
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################


# Callback to filter data table using CSV data
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-type-filter', 'value'),
     Input('outcome-type-filter', 'value')]
)
def update_table(rescue_type, outcome_type):
    filtered_df = df.copy()  # Use CSV data

    # Mapping rescue types to breed lists
    rescue_type_map = {
        'Water Rescue': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'],
        'Mountain Rescue': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog'],
        'Disaster Rescue': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever']
    }

    # Filter by rescue type if not reset
    if rescue_type != 'reset' and rescue_type in rescue_type_map:
        filtered_df = filtered_df[filtered_df['breed'].isin(rescue_type_map[rescue_type])]

    # Filter by outcome type if not 'All'
    if outcome_type != 'All':
        filtered_df = filtered_df[filtered_df['outcome_type'] == outcome_type]

    return filtered_df.to_dict('records')


# Highlight selected column(s)
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Update geolocation map based on selected row
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'selected_rows')]
)
def update_map(viewData, selected_rows):
    if viewData is None or len(viewData) == 0:
        return dl.Map(style={'width': '1000px', 'height': '500px'},
                      center=[30.75, -97.48], zoom=10, children=[dl.TileLayer()])

    dff = pd.DataFrame.from_dict(viewData)
    row = selected_rows[0] if selected_rows else 0

    if dff.shape[1] < 15:
        return dl.Map(style={'width': '1000px', 'height': '500px'},
                      center=[30.75, -97.48], zoom=10, children=[dl.TileLayer()])

    try:
        lat = float(dff.iloc[row, 13])
        lon = float(dff.iloc[row, 14])
    except (ValueError, TypeError):
        return dl.Map(style={'width': '1000px', 'height': '500px'},
                      center=[30.75, -97.48], zoom=10, children=[dl.TileLayer()])

    breed = dff.iloc[row, 4] if dff.shape[1] > 4 else "Unknown Breed"
    name = dff.iloc[row, 9] if dff.shape[1] > 9 else "Unknown Name"

    return dl.Map(style={'width': '1000px', 'height': '500px'},
                  center=[lat, lon], zoom=13, children=[
                      dl.TileLayer(id="base-layer-id"),
                      dl.Marker(position=[lat, lon], children=[
                          dl.Tooltip(breed),
                          dl.Popup([html.H1("Animal Name"), html.P(name)])
                      ])
                  ])

# Update bar chart based on filtered data
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'data')]
)
def update_graph(data):
    if not data:
        return html.Div(["No data to display."])

    dff = pd.DataFrame(data)

    if 'breed' not in dff.columns:
        return html.Div(["No breed data available."])

    breed_counts = dff['breed'].value_counts().nlargest(10).reset_index()
    breed_counts.columns = ['breed', 'count']

    fig = px.bar(
        breed_counts,
        x='breed',
        y='count',
        title='Top 10 Breeds in Filtered Results',
        labels={'breed': 'Breed', 'count': 'Number of Animals'}
    )

    return dcc.Graph(figure=fig)

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)



Dash app running on http://127.0.0.1:27631/
